In [ ]:
install.packages("sjlabelled")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘insight’, ‘datawizard’




In [ ]:
library(dplyr)
library(readr)
# library(ggplot2)
library(tidyr)

library(stringr)

library(dplyr)

library(sjlabelled)
# library(ggrepel)
# library(scales)
# library(ggpubr)
# library(plm)
# library(lmtest)


Attaching package: ‘sjlabelled’


The following object is masked from ‘package:ggplot2’:

    as_label


The following object is masked from ‘package:dplyr’:

    as_label




In [ ]:
# Temporary file and path
tfile_path <- tempfile()
tdir_path <- tempdir()

# Download zip file
download.file("http://davidcard.berkeley.edu/data_sets/njmin.zip",
              destfile = tfile_path)

# Unzip
unzip(tfile_path, exdir = tdir_path)

# Read codebook
codebook <- read_lines(file = paste0(tdir_path, "/codebook"))

# Generate a vector with variable names
variable_names <- codebook %>%
  `[`(8:59) %>% # Variablennamen starten bei Element 8 (sheet)
  `[`(-c(5, 6, 13, 14, 32, 33)) %>% # Elemente ohne Variablennamen entfernen
  str_sub(1, 8) %>% # längster Variablenname enthält 8 Zeichen
  str_squish() %>% # Whitespaces entfernen
  str_to_lower() # nur Kleinbuchstaben verwenden

# Generate a vector with variable labels
variable_labels <- codebook %>%
  `[`(8:59) %>% # variable names start at element 8 (sheet)
  `[`(-c(5, 6, 13, 14, 32, 33)) %>% # remove elements w/o variable names
  sub(".*\\.[0-9]", "", .) %>%
  `[`(-c(5:10))  %>% # these elements are combined later on
  str_squish() # remove white spaces

# Region
variable_labels[41] <- "region of restaurant"

# Read raw data
data_raw <- read_table(paste0(tdir_path, "/public.dat"),
                        col_names = FALSE)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double(),
  X10 = col_double(),
  X11 = col_double(),
  X18 = col_double(),
  X20 = col_double(),
  X21 = col_double(),
  X22 = col_double(),
  X28 = col_double(),
  X29 = col_double(),
  X30 = col_double(),
  X47 = col_logical()
)
ℹ Use `spec()` for the full column specifications.



In [ ]:
variable_labels

[1] "sheet number (unique store id)"        
 [2] "chain 1=bk; 2=kfc; 3=roys; 4=wendys"   
 [3] "1 if company owned"                    
 [4] "1 if NJ; 0 if Pa"                      
 [5] "number of call-backs*"                 
 [6] "# full-time employees"                 
 [7] "# part-time employees"                 
 [8] "# managers/ass't managers"             
 [9] "starting wage ($/hr)"                  
[10] "months to usual first raise"           
[11] "usual amount of first raise ($/hr)"    
[12] "1 if cash bounty for new workers"      
[13] "% employees affected by new minimum"   
[14] "free/reduced price code (See below)"   
[15] "hour of opening"                       
[16] "number hrs open per day"               
[17] "price of medium soda, including tax"   
[18] "price of small fries, including tax"   
[19] "price of entree, including tax"        
[20] "number of cash registers in store"     
[21] "number of registers open at 11:00 am"  
[22] "type 2nd interview 1=phone; 2=personal"
[23] "status of second interview: see below" 
[24] "date of second interview MMDDYY format"
[25] "number of call-backs*"                 
[26] "# full-time employees"                 
[27] "# part-time employees"                 
[28] "# managers/ass't managers"             
[29] "starting wage ($/hr)"                  
[30] "months to usual first raise"           
[31] "usual amount of first raise ($/hr)"    
[32] "1 if special program for new workers"  
[33] "free/reduced price code (See below)"   
[34] "hour of opening"                       
[35] "number hrs open per day"               
[36] "price of medium soda, including tax"   
[37] "price of small fries, including tax"   
[38] "price of entree, including tax"        
[39] "number of cash registers in store"     
[40] "number of registers open at 11:00 am"  
[41] "region of restaurant"

In [ ]:
# Add variable names
data_mod <- data_raw %>%
  select(-X47) %>% # remove empty column
  `colnames<-`(., variable_names) %>% # Assign variable names
  mutate_all(as.numeric) %>% # treat all variables as numeric
  mutate(sheet = ifelse(sheet == 407 & chain == 4, 408, sheet)) # duplicated sheet id 407

# Process data (currently wide format)
data_mod <- data_mod %>%
  # chain value label
  mutate(chain = case_when(chain == 1 ~ "bk",
                           chain == 2 ~ "kfc",
                           chain == 3 ~ "roys",
                           chain == 4 ~ "wendys")) %>%
  # state value label
  mutate(state = case_when(state == 1 ~ "New Jersey",
                           state == 0 ~ "Pennsylvania")) %>%
  # Region dummy
  mutate(region = case_when(southj == 1 ~ "southj",
                            centralj == 1 ~ "centralj",
                            northj == 1 ~ "northj",
                            shore == 1 ~ "shorej",
                            pa1 == 1 ~ "phillypa",
                            pa2 == 1 ~ "eastonpa")) %>%
  # meals value label
  mutate(meals = case_when(meals == 0 ~ "none",
                           meals == 1 ~ "free meals",
                           meals == 2 ~ "reduced price meals",
                           meals == 3 ~ "both free and reduced price meals")) %>%
  # meals value label
  mutate(meals2 = case_when(meals2 == 0 ~ "none",
                            meals2 == 1 ~ "free meals",
                            meals2 == 2 ~ "reduced price meals",
                            meals2 == 3 ~ "both free and reduced price meals")) %>%
  # status2 value label
  mutate(status2 = case_when(status2 == 0 ~ "refused second interview",
                             status2 == 1 ~ "answered 2nd interview",
                             status2 == 2 ~ "closed for renovations",
                             status2 == 3 ~ "closed permanently",
                             status2 == 4 ~ "closed for highway construction",
                             status2 == 5 ~ "closed due to Mall fire")) %>%

  mutate(co_owned = if_else(co_owned == 1, "yes", "no")) %>%
  mutate(bonus = if_else(bonus == 1, "yes", "no")) %>%
  mutate(special2 = if_else(special2 == 1, "yes", "no")) %>%
  mutate(type2 = if_else(type2 == 1, "phone", "personal")) %>%
  select(-southj, -centralj, -northj, -shore, -pa1, -pa2) %>% # now included in region dummy
  mutate(date2 = lubridate::mdy(date2)) %>% # Convert date
  rename(open2 = open2r) %>% #Fit name to wave 1
  rename(firstinc2 = firstin2) %>% # Fit name to wave 1
  sjlabelled::set_label(variable_labels) # Add stored variable labels

Warning message:
“There were 28 warnings in `mutate()`.
The first warning was:
ℹ In argument: `empft = .Primitive("as.double")(empft)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 27 remaining warnings.”


In [ ]:
# Structural variables
structure <- data_mod %>%
  select(sheet, chain, co_owned, state, region)

# Wave 1 variables
wave1 <- data_mod %>%
  select(-ends_with("2"), - names(structure)) %>%
  mutate(observation = "February 1992") %>%
  bind_cols(structure)

# Wave 2 variables
wave2 <- data_mod %>%
  select(ends_with("2")) %>%
  rename_all(~str_remove(., "2"))  %>%
  mutate(observation = "November 1992") %>%
  bind_cols(structure)

# Final dataset
card_krueger_1994 <- bind_rows(wave1, wave2) %>%
  select(sort(names(.))) %>% # Sort columns alphabetically
  sjlabelled::copy_labels(data_mod) # Restore variable labels

In [ ]:
card_krueger_1994_mod <- card_krueger_1994 %>%
  mutate(emptot = empft + nmgrs + 0.5 * emppt,
         pct_fte = empft / emptot * 100)

In [ ]:
card_krueger_1994_mod

bonus,chain,co_owned,date,empft,emppt,firstinc,hrsopen,inctime,meals,⋯,psoda,region,sheet,special,state,status,type,wage_st,emptot,pct_fte
<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
yes,bk,no,NA,30.0,15.0,NA,16.5,19,reduced price meals,⋯,1.03,phillypa,46,NA,Pennsylvania,NA,NA,NA,40.50,74.074074
no,kfc,no,NA,6.5,6.5,NA,13.0,26,reduced price meals,⋯,1.01,phillypa,49,NA,Pennsylvania,NA,NA,NA,13.75,47.272727
no,kfc,yes,NA,3.0,7.0,0.37,10.0,13,reduced price meals,⋯,0.95,phillypa,506,NA,Pennsylvania,NA,NA,NA,8.50,35.294118
yes,wendys,yes,NA,20.0,20.0,0.10,12.0,26,reduced price meals,⋯,0.87,phillypa,56,NA,Pennsylvania,NA,NA,5.00,34.00,58.823529
yes,wendys,yes,NA,6.0,26.0,0.15,12.0,52,both free and reduced price meals,⋯,0.87,phillypa,61,NA,Pennsylvania,NA,NA,5.50,24.00,25.000000
no,wendys,yes,NA,0.0,31.0,0.07,12.0,26,reduced price meals,⋯,0.87,phillypa,62,NA,Pennsylvania,NA,NA,5.00,20.50,0.000000
no,bk,no,NA,50.0,35.0,0.10,18.0,26,reduced price meals,⋯,1.04,eastonpa,445,NA,Pennsylvania,NA,NA,5.00,70.50,70.921986
no,bk,no,NA,10.0,17.0,0.25,24.0,52,reduced price meals,⋯,1.05,eastonpa,451,NA,Pennsylvania,NA,NA,5.00,23.50,42.553191
no,kfc,yes,NA,2.0,8.0,0.25,10.0,13,free meals,⋯,0.73,eastonpa,455,NA,Pennsylvania,NA,NA,5.25,11.00,18.181818


In [ ]:
write.csv(card_krueger_1994_mod, "card_krueger_1994_mod.csv")

In [2]:
install.packages("scales", repos="https://cran.cnr.berkeley.edu")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“unable to access index for repository https://cran.cnr.berkeley.edu/src/contrib:
  cannot open URL 'https://cran.cnr.berkeley.edu/src/contrib/PACKAGES'”
Warning message:
“package ‘scales’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [3]:
library(scales)

set.seed(650)
Attendance1 = rbeta( 1000, shape1 = 7, shape2 = 2,  ncp = 0 )
Attendance = rescale(Attendance1, to = c(20,100))

Treatment = Attendance
Treatment = ifelse(Treatment>median(Treatment), 0, 1)

Attendance_c = Attendance - median(Attendance)

err = rnorm(1000, 600, 100)

Y = 40 + 120*Treatment + 20*Attendance_c + err

Performance = rescale(Y, to = c(0,100))

data = as.data.frame(cbind(Performance, Treatment, Attendance, Attendance_c))

In [10]:
install.packages("stargazer")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [12]:
install.packages("pander")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘Rcpp’




In [11]:
library( stargazer )
library( dplyr )
library( pander )


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: ignored

In [4]:
data

Performance,Treatment,Attendance,Attendance_c
<dbl>,<dbl>,<dbl>,<dbl>
77.70212,0,88.96310,9.7036967
76.42701,0,88.84251,9.5831070
61.50637,1,62.68434,-16.5750698
76.46593,0,83.40147,4.1420677
87.25544,0,94.56779,15.3083834
67.50757,1,72.99882,-6.2605804
64.18186,1,73.58743,-5.6719750
70.15259,0,79.76771,0.5083037
63.33457,1,66.31617,-12.9432372


In [5]:
write.csv(data, "attendance_performance.csv")

In [7]:
library(stargazer)

mean( data$Performance[ data$Treatment==1 ] )

mean( data$Performance[ data$Treatment==0 ] )

reg <- lm( Performance ~ Treatment +
             Attendance_c, data = data )

stargazer( reg,
           type = "html",
           dep.var.labels = ("Final exam grade"),
           column.labels = c(""),
           covariate.labels = c("Treatment", "Attendance"),
           omit.stat = "all",
           digits = 2 )

reg <- lm( Performance ~ Treatment + Attendance, data = data )

stargazer( reg,
           type = "html",
           dep.var.labels = ("Final exam grade"),
           column.labels = c(""),
           covariate.labels = c("Treatment", "Attendance"),
           omit.stat = "all",
           digits = 2 )

# We first predict the final exam grades based on our regression discontinuity model.
predfit <- predict( reg, data )

# We set colors for the graph
palette( c( adjustcolor( "steelblue", alpha.f=0.3), adjustcolor( "darkred", alpha.f=0.3)  ) )

# To differently color the treatment and control group we need to create a new Treatment variable that where the Control group = 2.
data$Treatment2 <- ifelse( data$Treatment==0, 2, 1 )

# Plot our observed value
plot( Attendance_c, Performance, cex=2,
      col=data$Treatment2, pch=19, bty="n",
      xlab="Attendance Rate (centered)",
      ylab="Final exam grade")

# Plot a line at the cutoff point
abline( v=0, col="red", lwd=2, lty=2 )

# Plot our predicted values
points( Attendance_c, predfit, col="black", cex=0.8, pch = 19 )

## Add a legend
points( 10, 26, col="steelblue", pch=19, cex=2 )
text( 10, 26, "Treatment group", col="steelblue", pos=4 )
points( 10, 20, col="darkred", pch=19, cex=2 )
text( 10, 20, "Control group", col="darkred", pos=4 )

## We create a new dataset where the variable Treatment is only equal to 0 as no student has been part of the intervention
data2 <- data.frame( Performance=data$Performance,
                     Treatment=rep(0),
                     Attendance_c=data$Attendance_c )

## Based on our previous model we predict the final grades, had the intervention not occured
predfit2 <- predict( reg, data2 )

data2$predfit2 <- predfit2

## We set a new color for representing the counterfactual
counterfactual <- adjustcolor("darkorange2", alpha.f=0.2 )

## Plot the observed valies
plot( Attendance_c, Performance, cex=2,
      col=data$Treatment2, pch=19, bty="n",
      xlab="Attendance Rate (centered)",
      ylab="Final exam grade")

# Plot a line at the cutoff
abline( v=0, col="red", lwd=2, lty=2 )

# Plot the counterfactual
# Note that we add a bit of noise in the data to facilitate visualization
points( Attendance_c[ Attendance_c < 0 ],
        predfit2[ Attendance_c < 0 ] + rnorm(500,0,5),
        col=counterfactual, lwd=2, pch = 19, cex = 2 )

# Add a legend
## Add a legend
points( 10, 20, col="steelblue", pch=19, cex=2 )
text( 10, 20, "Treatment group", col="steelblue", pos=4 )
points( 10, 26, col="darkred", pch=19, cex=2 )
text( 10, 26, "Control group", col="darkred", pos=4 )
points( 10, 14, col="darkorange2", pch=19, cex=2 )
text( 10, 14, "Counterfactual", col="darkorange2", pos=4 )

ERROR: ignored